In [17]:
from transformers import BertTokenizer
from razdel import sentenize
from models.model_builder import AbsSummarizer
import torch
import numpy as np
import pandas as pd
import tqdm
import json
import pickle
import os

AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.activations' has no attribute 'swish'

In [2]:
DEVICE = 'cuda'

In [11]:
import os
from tqdm import trange

class TelegramPredictionsDataset:
    def __init__(self, jsonl_path):
        data = pd.read_json(jsonl_path, lines=True)
        self.texts = []
        self.titles = []
        self.target_vectors = []
        for i in trange(len(data)):
            text = data.iloc[i]['text'].lower().replace('\xa0', ' ').replace('\n', ' ').strip()
            title = data.iloc[i]['title'].lower()

            if not text or not title or text.count(' ') < 3 or title.count(' ') < 3:
                continue
                
            self.texts.append(text)
            self.titles.append(title)
            self.target_vectors.append(doc2vec(text + " " + title, BertSumAbs, mode="FirstCLS"))
            
        self.target_vectors = torch.FloatTensor(self.target_vectors)
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, idx):
        return self.texts[idx], self.titles[idx], self.target_vectors[idx]

### Reading dataset

In [16]:
with open('/data/alolbuhtijarov/datasets/BertSumAbs_predictions/vectors.npy', 'rb') as fh:
    fsz = os.fstat(fh.fileno()).st_size
    out = np.load(fh).reshape(1, -1)
    a = 0
    while fh.tell() < fsz:
        out = np.concatenate((out, np.load(fh).reshape(1, -1)), axis=0)
        a += 1
        if a == 20:
            break
        
out.shape

NameError: name 'os' is not defined

In [13]:
b = pd.read_json("texts.jsonl",encoding='utf-8',
                    lines=True, chunksize=CHINK_SIZE)
b

### Encoder with pretrained FastText embeddings

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        vec = list(map(float, tokens[1:]))
        vec = np.array(vec)
        data[tokens[0]] = vec
    return data

In [ ]:
ft_vecs = load_vectors("/data/alolbuhtijarov/wiki-news-300d-1M-subword.vec")

In [ ]:
vocab_token_vectors = torch.FloatTensor([
    ft_vecs.get(w) if w in ft_vecs else np.random.rand(300) * 1e-3 for w in tokens
])

vocab_token_vectors.shape

In [ ]:
class SmallEncoder(nn.Module):
    def __init__(self, n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=128,
                freeze_embed=True):
        super().__init__()
        
        self.embed = nn.Embedding.from_pretrained(vocab_token_vectors, freeze=freeze_embed)

        self.layer_title = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU(),
        )

        self.layer_text = nn.Sequential(
            nn.Conv1d(in_channels=300, out_channels=300, kernel_size=3),
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.BatchNorm1d(num_features=300),
            nn.ReLU()
        )

        self.layer_cat = nn.Sequential(
            nn.Linear(n_cat_features, hid_size * 2),
            nn.BatchNorm1d(num_features=hid_size * 2),
            nn.ReLU(),
            nn.Linear(hid_size * 2, hid_size),
            nn.BatchNorm1d(num_features=hid_size),
            nn.ReLU()
        )

        self.predictor = nn.Linear(300 * 2 + hid_size, 1)


    def forward(self, batch):
        title = self.embed(batch["Title"])
        title = title.permute(0, 2, 1)
        title = self.layer_title(title).squeeze(-1)

        text = self.embed(batch["FullDescription"])
        text = text.permute(0, 2, 1)
        text = self.layer_text(text).squeeze(-1)

        cat = self.layer_cat(batch["Categorical"])

        x = torch.cat([title, text, cat], dim=1)
        return self.predictor(x).squeeze(-1)

https://pytorch.org/docs/stable/optim.html#per-parameter-options

In [ ]:
def separate_optimizer(net):
    embed_param = [kv[1] for kv in net.named_parameters() if kv[0] == 'embed.weight']
    model_params = [kv[1] for kv in net.named_parameters() if kv[0] != 'embed.weight']
    opt = torch.optim.Adam([
                {'params': model_params},
                {'params': embed_param, 'lr': 3e-4}
    ], lr=3e-3)
    return opt